# load package and model

In [1]:
import os
import sys
CONFIG_AI_PATH=os.environ["CONFIG_AI_PATH"]
if CONFIG_AI_PATH not in sys.path:
    sys.path.insert(0, CONFIG_AI_PATH)
sys.path




from config_ai.models.text_classify import CLSTokenClassifyModel
# from config_ai.evaluate import eval_text_classify
# from config_ai.models.text_classify.common import *
# from config_ai.data_utils import *
# from config_ai.backend import set_tf_config
# from config_ai.models import load_model
from config_ai.experiments import get_model_config
from config_ai.utils import read_config
from snippets.utils import *

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# set_tf_config()


['/Users/chenhao/workspace/ConfigAI',
 '/Users/chenhao/workspace/ConfigAI/notebooks/text_classify',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Users/chenhao/workspace/ConfigAI/venv/lib/python3.8/site-packages']

In [2]:
config_path = f"{CONFIG_AI_PATH}/examples/text_classify/sentiment_cls_token_classify.ini"

config = read_config(config_path)
config
model_config = get_model_config(config)
# model_config

model = CLSTokenClassifyModel(config=model_config)
model.id2label

2022-08-08 18:13:35 [INFO][utils.py:75]:parsing config with path:/Users/chenhao/workspace/ConfigAI/examples/text_classify/sentiment_cls_token_classify.ini
2022-08-08 18:13:35 [INFO][utils.py:90]:loading base config...
2022-08-08 18:13:35 [INFO][utils.py:75]:parsing config with path:/Users/chenhao/workspace/ConfigAI/examples/base_config.ini


{'common_config': {'experiment_dir': '/Users/chenhao/experiment',
  'project_name': 'config_ai_example',
  'is_train': True,
  'is_test': True,
  'is_save': True,
  'save_args': {'fmt': 'h5'},
  'eval_phase_list': ['train', 'dev'],
  'output_phase_list': ['train', 'dev', 'test'],
  'is_overwrite_experiment': True,
  'default_random_seed': 10,
  'base_config': '/Users/chenhao/workspace/ConfigAI/examples/base_config.ini',
  'model_cls': 'CLSTokenClassifyModel',
  'model_name': 'cls_token_classify_example'},
 'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/Users/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'nn_model_config': {'pretrained_model_tag': 'bert',
  'pretrained_model_path': '/Users/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'},
 'compile_config': {'optimizer_name': 'adam',
  'optimizer_args': {'learning_rate': 3e-05}},
 'train_config': {'epochs': 2,
  'batch_size': 32,
  'overwrite_cache': False,
  'steps_per_epoch': 100

2022-08-08 18:13:35 [INFO][core.py:45]:loading config
2022-08-08 18:13:35 [INFO][core.py:46]:init model with config:


{0: '正向', 1: '中性', 2: '负向'}

# process data

In [6]:
train_data_path = config['data_config']['train_data_path']
train_data = model.load_examples(train_data_path)
f"{train_data_path}: {len(train_data)} items"


dev_data_path = config['data_config']['dev_data_path']
dev_data = model.load_examples(dev_data_path)
f"{dev_data_path}: {len(dev_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.load_examples(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'/Users/chenhao/workspace/ConfigAI/data/sentiment_classify/labeled.jsonl: 3 items'

'/Users/chenhao/workspace/ConfigAI/data/sentiment_classify/labeled.jsonl: 3 items'

'/Users/chenhao/workspace/ConfigAI/data/sentiment_classify/labeled.jsonl: 3 items'

TextExample(content='这家酒店真不错')

In [5]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [6]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '这家酒店真不错',
 'text': '这家酒店真不错',
 'extra_text': None,
 'token_ids': [101, 6821, 2157, 6983, 2421, 4696, 679, 7231, 102],
 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['[CLS]', '这', '家', '酒', '店', '真', '不', '错', '[SEP]'],
 'labels': ['正向']}

In [7]:
records = data_manager.get_records(mode="train")
next(records)



{'idx': 0,
 'full_text': '这家酒店真不错',
 'text': '这家酒店真不错',
 'extra_text': None,
 'token_ids': [101, 6821, 2157, 6983, 2421, 4696, 679, 7231, 102],
 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['[CLS]', '这', '家', '酒', '店', '真', '不', '错', '[SEP]'],
 'labels': ['正向'],
 'classify_output': 0}

# build&compile model

In [8]:
nn_model_args = dict(**config["nn_model_config"])
nn_model_args.update(transformer_kwargs=dict(dropout_rate=0.3))
nn_model_args

model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12',
 'transformer_kwargs': {'dropout_rate': 0.3}}

2021-11-23 14:45:42.327470: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-23 14:45:42.337630: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3597740000 Hz
2021-11-23 14:45:42.338341: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5566e1621300 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-23 14:45:42.338370: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-23 14:45:42.673919: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5566e16bb810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-23 14:45:42.673951: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2021-11-23 14:45:42.675245: I tensorflow

In [9]:
compile_args = dict(**config["compile_config"])
# compile_args.update(rdrop_alpha=4)
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-11-23 14:45:46 [INFO][cls_token_classify.py:82]:compiling model...
2021-11-23 14:45:46 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-11-23 14:45:47 [INFO][losses.py:101]:build loss layer with loss function:<function sparse_categorical_crossentropy at 0x7f902d0b9ca0>
2021-11-23 14:45:47 [INFO][cls_token_classify.py:106]:training model's summary:
2021-11-23 14:45:47 [INFO][layer_utils.py:192]:Model: "train_model"
2021-11-23 14:45:47 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-23 14:45:47 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-11-23 14:45:47 [INFO][layer_utils.py:195]:==================================================================================================
2021-11-23 14:45:47 [INFO][layer_utils.py:190]:token_ids (InputLayer)          [(None, None)]       0                    

In [10]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


[<tf.Tensor 'token_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'segment_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'classify_output:0' shape=(None,) dtype=int32>]

[<tf.Tensor 'nn_model/Identity:0' shape=(None, 3) dtype=float32>]

[<tf.Tensor 'loss_layer/Identity:0' shape=(None,) dtype=float32>]

In [11]:
model.get_dataset_info("train")
model.get_dataset_info("test")

({'token_ids': 'int32', 'segment_ids': 'int32', 'classify_output': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'classify_output': ()})

({'token_ids': 'int32', 'segment_ids': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,)})

In [12]:
dataset = data_manager.get_train_dataset(batch_size=1)

item = next(dataset.as_numpy_iterator())
item


{'token_ids': array([[ 101, 6821, 2157, 6983, 2421, 4696,  679, 7231,  102]],
       dtype=int32),
 'segment_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32),
 'classify_output': array([0], dtype=int32)}

# train_model

In [13]:
train_args = dict(**config["train_config"])
train_args.update(steps_per_epoch=100, batch_size=8, verbose=1)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 2,
 'batch_size': 8,
 'overwrite_cache': False,
 'steps_per_epoch': 100,
 'verbose': 1}

100%|██████████| 3/3 [00:00<00:00, 4919.04it/s]
2021-11-23 14:45:52 [INFO][tf_core.py:173]:train on 3 tensors


Epoch 1/2


2021-11-23 14:45:56.202100: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


100/100 [==============================] - 13s 130ms/step - loss: 0.1805 - accuracy: 0.9675
Epoch 2/2
100/100 [==============================] - 12s 124ms/step - loss: 0.0043 - accuracy: 1.0000


2021-11-23 14:46:22 [INFO][tf_core.py:189]:training finished


# predict&evaluate

In [14]:
to_pred = train_data[:4]
to_pred

pred = model.predict(to_pred, show_detail=True)
pred[:4]


[LabeledTextClassifyExample(text='这家酒店真不错', label=Label(name='正向', prob=1.0)),
 LabeledTextClassifyExample(text='这门课太差了', label=Label(name='负向', prob=1.0)),
 LabeledTextClassifyExample(text='这是一家酒店', label=Label(name='中性', prob=1.0))]

100%|██████████| 3/3 [00:00<00:00, 6064.05it/s]
2021-11-23 14:48:07 [INFO][tf_core.py:204]:predicting with tf model...
1it [00:00, 15.67it/s]
2021-11-23 14:48:08 [INFO][decorators.py:28]:function:_model_predict cost:0.066 seconds
2021-11-23 14:48:08 [INFO][decorators.py:28]:function:_post_predict cost:0.002 seconds


[Label(name='正向', prob=0.9997114539146423),
 Label(name='负向', prob=0.999711811542511),
 Label(name='中性', prob=0.9997040629386902)]

In [15]:
output_data = get_text_classify_output(to_pred, pred)
output_data[:4]

[{'text': '这家酒店真不错',
  'label': {'name': '正向', 'prob': 1.0},
  'predict': {'name': '正向', 'prob': 0.9997114539146423},
  'tp_set': {'正向'},
  'fp_set': set(),
  'fn_set': set()},
 {'text': '这门课太差了',
  'label': {'name': '负向', 'prob': 1.0},
  'predict': {'name': '负向', 'prob': 0.999711811542511},
  'tp_set': {'负向'},
  'fp_set': set(),
  'fn_set': set()},
 {'text': '这是一家酒店',
  'label': {'name': '中性', 'prob': 1.0},
  'predict': {'name': '中性', 'prob': 0.9997040629386902},
  'tp_set': {'中性'},
  'fp_set': set(),
  'fn_set': set()}]

In [16]:
true_labels = [e.label for e in to_pred]
eval_rs = eval_text_classify(true_labels, pred)
eval_rs

{'detail': {'中性': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  '正向': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  '负向': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0}},
 'micro': {'tp': 3,
  'fp': 0,
  'fn': 0,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0},
 'macro': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}

# save&load

In [17]:
model_path = os.path.join(config["common_config"]["experiment_dir"],
                          config["common_config"]["project_name"],
                          config["common_config"]["model_name"],"model")

save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model'

2021-11-23 14:48:11 [INFO][core.py:79]:saving model to /nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model
2021-11-23 14:48:11 [INFO][tf_core.py:55]:saving keras model to path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model/nn_model/nn_model.h5
2021-11-23 14:48:13 [INFO][core.py:144]:save model done


In [18]:
loaded_model = load_model(path=model_path)
loaded_model

2021-11-23 14:48:13 [INFO][core.py:90]:loading model from path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model
2021-11-23 14:48:13 [INFO][core.py:47]:init model with config:
2021-11-23 14:48:13 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-23 14:48:13 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs
2021-11-23 14:48:13 [INFO][tf_core.py:157]:loading keras model from path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model/nn_model/nn_model.h5 with format:h5
2021-11-23 14:48:17 [INFO][layer_utils.py:192]:Model: "nn_model"
2021-11-23 14:48:17 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-23 14:48:17 [INFO][layer_utils.py:190]:Layer (type)                   

In [20]:
loaded_pred = loaded_model.predict(data=to_pred)
loaded_pred

100%|██████████| 3/3 [00:00<00:00, 11285.12it/s]
2021-11-22 15:05:23 [INFO][tf_core.py:204]:predicting with tf model...
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_model_predict cost:0.036 seconds
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_post_predict cost:0.001 seconds


[Label(name='正向', prob=0.9998229146003723),
 Label(name='负向', prob=0.9997743964195251),
 Label(name='中性', prob=0.9997645020484924)]